#             best_acc = soul.evolve(X_evo_v, y_evo_v, generations=50)


Increase gen for Stability and accuracy.

In [ ]:
import numpy as np
import pandas as pd
import warnings
import random
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, PowerTransformer, StandardScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import RidgeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, accuracy_score
from scipy.optimize import minimize
from scipy.fft import fft
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# GPU CHECK
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ GPU NOT FOUND: Running in Slow Mode")

warnings.filterwarnings('ignore')

# --- 1. THE HOLOGRAPHIC SOUL (Unit 3 - Multiverse Edition) ---
class HolographicSoulUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=15):
        self.k = k
        self.dna_ = {
            'freq': 2.0, 'gamma': 0.5, 'power': 2.0,
            'metric': 'minkowski', 'p': 2.0,
            'phase': 0.0, 'dim_reduction': 'none'
        }
        self.projector_ = None
        self.X_raw_source_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self._apply_projection(X)
        self.y_train_ = y
        return self

    def _apply_projection(self, X):
        if self.dna_['dim_reduction'] == 'holo':
            n_components = max(2, int(np.sqrt(X.shape[1])))
            self.projector_ = GaussianRandomProjection(n_components=n_components, random_state=42)
            self.X_train_ = self.projector_.fit_transform(X)
        elif self.dna_['dim_reduction'] == 'pca':
             n_components = max(2, int(np.sqrt(X.shape[1])))
             self.projector_ = PCA(n_components=n_components, random_state=42)
             self.X_train_ = self.projector_.fit_transform(X)
        else:
            self.projector_ = None
            self.X_train_ = X

    # [FIX] Indentation corrected: Now this method is part of the class
    def set_raw_source(self, X):
        self.X_raw_source_ = X

    def evolve(self, X_val, y_val, generations=1000):
        n_universes = 10
        best_acc = self.score(X_val, y_val)
        best_dna = self.dna_.copy()

        # Smart Init
        if GPU_AVAILABLE:
            sample_X = cp.asarray(self.X_train_[:100])
            dists = cp.mean(cp.linalg.norm(sample_X[:, None, :] - sample_X[None, :, :], axis=2))
            median_dist = float(cp.asnumpy(dists))
        else:
            median_dist = 1.0

        if median_dist > 0:
            best_dna['freq'] = 3.14159 / median_dist

        for i in range(generations):
            candidates = []
            for _ in range(n_universes):
                mutant = best_dna.copy()
                trait = random.choice(list(mutant.keys()))
                if trait == 'freq': mutant['freq'] *= np.random.uniform(0.8, 1.25)
                elif trait == 'gamma': mutant['gamma'] = np.random.uniform(0.1, 5.0)
                elif trait == 'power': mutant['power'] = random.choice([0.5, 1.0, 2.0, 3.0, 4.0, 6.0])
                elif trait == 'p': mutant['p'] = np.clip(mutant['p'] + np.random.uniform(-0.5, 0.5), 0.5, 8.0)
                elif trait == 'phase': mutant['phase'] = np.random.uniform(0, 3.14159)
                elif trait == 'dim_reduction': mutant['dim_reduction'] = random.choice(['none', 'holo', 'pca'])
                candidates.append(mutant)

            generation_best_acc = -1
            generation_best_dna = None

            for mutant_dna in candidates:
                self.dna_ = mutant_dna
                if self.X_raw_source_ is not None: self._apply_projection(self.X_raw_source_)
                acc = self.score(X_val, y_val)
                if acc > generation_best_acc:
                    generation_best_acc = acc
                    generation_best_dna = mutant_dna

            if generation_best_acc >= best_acc:
                best_acc = generation_best_acc
                best_dna = generation_best_dna
            else:
                self.dna_ = best_dna
                if self.X_raw_source_ is not None: self._apply_projection(self.X_raw_source_)

        self.dna_ = best_dna
        return best_acc

    def predict_proba(self, X):
        if self.projector_ is not None: X_curr = self.projector_.transform(X)
        else: X_curr = X
        if GPU_AVAILABLE: return self._predict_proba_gpu(X_curr)
        else: return np.zeros((len(X), len(self.classes_)))

    def _predict_proba_gpu(self, X):
        X_tr_g = cp.asarray(self.X_train_, dtype=cp.float32)
        X_te_g = cp.asarray(X, dtype=cp.float32)
        y_tr_g = cp.asarray(self.y_train_)

        n_test = len(X_te_g)
        n_classes = len(self.classes_)
        probas = []
        batch_size = 256

        p_norm = self.dna_.get('p', 2.0)
        gamma = self.dna_['gamma']
        freq = self.dna_['freq']
        power = self.dna_['power']
        phase = self.dna_.get('phase', 0.0)

        for i in range(0, n_test, batch_size):
            end = min(i + batch_size, n_test)
            batch_te = X_te_g[i:end]
            diff = cp.abs(batch_te[:, None, :] - X_tr_g[None, :, :])
            dists = cp.sum(cp.power(diff, p_norm), axis=2)
            dists = cp.power(dists, 1.0/p_norm)
            top_k_idx = cp.argsort(dists, axis=1)[:, :self.k]
            row_idx = cp.arange(len(batch_te))[:, None]
            top_dists = dists[row_idx, top_k_idx]
            top_y = y_tr_g[top_k_idx]

            cosine_term = 1.0 + cp.cos(freq * top_dists + phase)
            cosine_term = cp.maximum(cosine_term, 0.0)
            w = cp.exp(-gamma * (top_dists**2)) * cosine_term
            w = cp.power(w, power)

            batch_probs = cp.zeros((len(batch_te), n_classes))
            for c_idx, cls in enumerate(self.classes_):
                class_mask = (top_y == cls)
                batch_probs[:, c_idx] = cp.sum(w * class_mask, axis=1)

            total_energy = cp.sum(batch_probs, axis=1, keepdims=True)
            total_energy[total_energy == 0] = 1.0
            batch_probs /= total_energy
            probas.append(batch_probs)
            del batch_te, dists, diff, top_k_idx, top_dists, w, cosine_term
            cp.get_default_memory_pool().free_all_blocks()

        return cp.asnumpy(cp.concatenate(probas))

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))


# --- 3. THE QUANTUM FIELD (Unit 4) ---
class QuantumFieldUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.rbf_feature_ = RBFSampler(n_components=100, random_state=42)
        self.classifier_ = RidgeClassifier(alpha=1.0)
        self.classes_ = None
        self.dna_ = {'gamma': 1.0, 'n_components': 100}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.rbf_feature_.set_params(gamma=self.dna_['gamma'], n_components=self.dna_['n_components'])
        X_quantum = self.rbf_feature_.fit_transform(X)
        self.classifier_.fit(X_quantum, y)
        return self

    def predict_proba(self, X):
        X_quantum = self.rbf_feature_.transform(X)
        d = self.classifier_.decision_function(X_quantum)
        if len(self.classes_) == 2:
            probs = 1 / (1 + np.exp(-d))
            return np.column_stack([1-probs, probs])
        else:
            exp_d = np.exp(d - np.max(d, axis=1, keepdims=True))
            return exp_d / np.sum(exp_d, axis=1, keepdims=True)

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])

# --- 4. THE ENTROPY MAXWELL (Unit 5) ---
# --- 4. THE ENTROPY MAXWELL (Unit 5) - STABILIZED ---
class EntropyMaxwellUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.models_ = {}
        self.classes_ = None
        self.priors_ = None
        self.dna_ = {'n_components': 1}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.models_ = {}
        self.priors_ = {}
        n_samples = len(y)
        for cls in self.classes_:
            X_c = X[y == cls]
            # Safety: If a class has too few samples, fallback
            if len(X_c) < 2:
                self.priors_[cls] = 0.0
                continue

            self.priors_[cls] = len(X_c) / n_samples
            n_comp = min(self.dna_['n_components'], len(X_c))

            # Added Reg_covar 1e-4 for better stability
            gmm = GaussianMixture(n_components=n_comp, covariance_type='full',
                                  reg_covar=1e-4, random_state=42)
            gmm.fit(X_c)
            self.models_[cls] = gmm
        return self

    def predict_proba(self, X):
        probs = np.zeros((len(X), len(self.classes_)))
        for i, cls in enumerate(self.classes_):
            if cls in self.models_:
                log_prob = self.models_[cls].score_samples(X)
                # Clip log_prob to prevent underflow/overflow
                log_prob = np.clip(log_prob, -100, 100)
                probs[:, i] = np.exp(log_prob) * self.priors_[cls]

        # STABILITY FIX: Add epsilon to avoid ZeroDivisionError (0/0 = NaN)
        total = np.sum(probs, axis=1, keepdims=True) + 1e-10
        return probs / total

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])

# --- 5. THE OMNI-KERNEL NEXUS (Unit 6 - Hybrid Kernel Evolution) ---
class OmniKernelUnit(BaseEstimator, ClassifierMixin):
    """
    Dynamically evolves the mathematical kernel to fit the universe of the data.
    Options: RBF (Sphere), Polynomial (Twist), Sigmoid (Switch), Linear (Plane).
    """
    def __init__(self):
        self.model_ = None
        self.classes_ = None
        self.dna_ = {
            'kernel': 'rbf',
            'C': 1.0,
            'gamma': 'scale',
            'degree': 3,
            'coef0': 0.0
        }

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        # Construct SVC based on DNA
        # Note: probability=True is expensive, but needed for Soft Voting.
        # In a real massive system, we'd use CalibratedClassifierCV + LinearSVC for speed.
        # For accuracy aim, we use standard SVC with probability.
        self.model_ = SVC(
            kernel=self.dna_['kernel'],
            C=self.dna_['C'],
            gamma=self.dna_['gamma'],
            degree=self.dna_['degree'],
            coef0=self.dna_['coef0'],
            probability=True,
            random_state=42,
            cache_size=500
        )
        self.model_.fit(X, y)
        return self

    def predict_proba(self, X):
        return self.model_.predict_proba(X)

    def score(self, X, y):
        return self.model_.score(X, y)


from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import NuSVC

import numpy as np
import pandas as pd
import warnings
import random
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.calibration import CalibratedClassifierCV
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import log_loss, accuracy_score
from sklearn.utils.validation import check_X_y
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

# --- 7. THE TITAN-14 "BEAST MODE" (Endgame Edition) ---
# --- 7. THE TITAN-17 "BEAST MODE" (AGI Edition) ---
class HarmonicResonanceClassifier_BEAST_17D(BaseEstimator, ClassifierMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose
        # Robust scaling with wider quantile to catch outliers
        self.scaler_ = RobustScaler(quantile_range=(15.0, 85.0))
        self.weights_ = None
        self.classes_ = None

        # --- THE 17 BEASTS (Maximum Fidelity + AGI Souls) ---

        # 1. LOGIC ALPHA (The Overlord - ExtraTrees)
        self.unit_01 = ExtraTreesClassifier(n_estimators=1000, bootstrap=False,
                                            max_features='sqrt', n_jobs=-1, random_state=42)

        # 2. LOGIC BETA (The Tactician - RandomForest)
        self.unit_02 = RandomForestClassifier(n_estimators=1000, criterion='gini',
                                              n_jobs=-1, random_state=42)

        # 3. LOGIC GAMMA (The Swarm - HistGradient)
        self.unit_03 = HistGradientBoostingClassifier(max_iter=500, learning_rate=0.05,
                                                      early_stopping=False, random_state=42)

        # 4. GRADIENT ALPHA (The Sniper - XGBoost Deep)
        self.unit_04 = XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.02,
                                     subsample=0.8, colsample_bytree=0.8,
                                     use_label_encoder=False, eval_metric='logloss',
                                     tree_method='hist', n_jobs=-1, random_state=42)

        # 5. GRADIENT BETA (The Nuke - XGBoost Fast)
        self.unit_05 = XGBClassifier(n_estimators=1000, max_depth=3, learning_rate=0.1,
                                     use_label_encoder=False, eval_metric='logloss',
                                     tree_method='hist', n_jobs=-1, random_state=42)

        # 6. KERNEL ALPHA (The Warp - NuSVC)
        self.unit_06 = NuSVC(nu=0.05, kernel='rbf', gamma='scale', probability=True, random_state=42)

        # 7. KERNEL BETA (The Manifold - Poly SVC)
        self.unit_07 = SVC(kernel='poly', degree=2, C=10.0, probability=True, random_state=42)

        # 8. GEOMETRY ALPHA (The Cluster - Euclidean)
        self.unit_08 = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='euclidean', n_jobs=-1)

        # 9. GEOMETRY BETA (The Field - Manhattan)
        self.unit_09 = KNeighborsClassifier(n_neighbors=9, weights='distance', metric='manhattan', n_jobs=-1)

        # 10. SPACETIME (The Metric - Mahalanobis equivalent via QDA)
        self.unit_10 = QuadraticDiscriminantAnalysis(reg_param=0.01)

        # 11. RESONANCE (The Wave - Calibrated Linear)
        self.unit_11 = CalibratedClassifierCV(LinearSVC(C=0.5, dual=False, max_iter=5000), cv=5)

        # --- THE SOULS (Standard) ---

        # 12. THE HOLOGRAPHIC SOUL (The Original - Logic Seed)
        self.unit_12 = HolographicSoulUnit(k=15)

        # 13. TWIN SOUL ALPHA (Mirror 1 - Chaos Seed)
        self.unit_13 = HolographicSoulUnit(k=15)

        # 14. TWIN SOUL BETA (Mirror 2 - Order Seed)
        self.unit_14 = HolographicSoulUnit(k=15)

        # --- THE AGI TRINITY (New Powerful Souls) ---

        # 15. SOUL D (AGI Level - Hyper-Connectivity)
        # Higher 'k' for denser information processing
        self.unit_15 = HolographicSoulUnit(k=25)

        # 16. SOUL E (AGI Level - Deep Resonance)
        self.unit_16 = HolographicSoulUnit(k=25)

        # 17. SOUL F (AGI Level - Universal Observer)
        self.unit_17 = HolographicSoulUnit(k=25)


    def fit(self, X, y):
        # 1. Validation and Scaling
        y = np.array(y).astype(int)
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        X_scaled = self.scaler_.fit_transform(X)

        # 2. Split for INTERNAL EVOLUTION (The Self-Talk Phase)
        X_evo_t, X_evo_v, y_evo_t, y_evo_v = train_test_split(
            X_scaled, y, test_size=0.2, stratify=y, random_state=42
        )

        if self.verbose:
            print("\n" + "!"*60)
            print(" >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<")
            print("!"*60)
            print(" > Phase 1: Awakening the Souls (Evolutionary Adaptation)...")

        # --- STEP A: FORCED EVOLUTION OF THE 6 SOULS ---

        soul_units = [
            ("SOUL-01 (Original)", self.unit_12),
            ("SOUL-02 (Mirror A)", self.unit_13),
            ("SOUL-03 (Mirror B)", self.unit_14),
            ("SOUL-D (AGI Hyper)", self.unit_15),
            ("SOUL-E (AGI Deep)",  self.unit_16),
            ("SOUL-F (AGI Omni)",  self.unit_17)
        ]

        for name, soul in soul_units:
            # We seed the raw source so they can do holographic projection
            soul.set_raw_source(X_evo_t)

            # FIT first to initialize structures
            soul.fit(X_evo_t, y_evo_t)

            # EVOLVE: This forces the DNA to change based on X_evo_v performance
            if self.verbose:
                print(f"   > {name} is meditating on the data...")

            # We pass the validation set so it knows if its mutations are working
            best_acc = soul.evolve(X_evo_v, y_evo_v, generations=50)

            if self.verbose:
                dna = soul.dna_
                print(f"      [EVOLVED] Acc: {best_acc:.2%} | Freq: {dna.get('freq',0):.2f} | Gamma: {dna.get('gamma',0):.2f}")

        # --- STEP B: FIT THE REST OF THE BEASTS ---
        if self.verbose:
            print(" > Phase 2: Deploying Logic & Physics Units...")

        other_units = [
            self.unit_01, self.unit_02, self.unit_03, self.unit_04,
            self.unit_05, self.unit_06, self.unit_07, self.unit_08,
            self.unit_09, self.unit_10, self.unit_11
        ]

        for i, unit in enumerate(other_units):
            try:
                unit.fit(X_evo_t, y_evo_t)
            except Exception as e:
                if self.verbose: print(f"   [Warning] Unit {i+1} adaptation issue: {e}")
                unit = SVC(kernel='rbf', C=1.0, probability=True)
                unit.fit(X_evo_t, y_evo_t)


        # --- STEP C: OPTIMIZATION (Finding the Perfect Consensus) ---
        if self.verbose:
            print(" > Phase 3: The Council of 17 (Weight Optimization)...")

        # Gather all units including the now-evolved souls
        all_units = other_units + [
            self.unit_12, self.unit_13, self.unit_14,
            self.unit_15, self.unit_16, self.unit_17
        ]

        n_units = len(all_units) # Should be 17

        # Get Predictions (Proba) on the validation set
        preds_proba = []
        for unit in all_units:
            try:
                if hasattr(unit, "predict_proba"):
                    p = unit.predict_proba(X_evo_v)
                else:
                    d = unit.decision_function(X_evo_v)
                    p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)
                preds_proba.append(p)
            except:
                preds_proba.append(np.ones((len(X_evo_v), n_classes)) / n_classes)

        # Optimization Function
        def loss_func(w):
            w = np.abs(w)
            w = w / np.sum(w)
            final_p = np.zeros_like(preds_proba[0])
            for k in range(n_units):
                final_p += w[k] * preds_proba[k]
            ll = log_loss(y_evo_v, np.clip(final_p, 1e-15, 1-1e-15))
            return ll

        # Smart Initialization
        accuracies = [accuracy_score(y_evo_v, np.argmax(p, axis=1)) for p in preds_proba]
        acc_pow = np.array(accuracies) ** 4
        init_weights = acc_pow / np.sum(acc_pow)

        res = minimize(
            loss_func,
            init_weights,
            bounds=[(0.0, 1.0)] * n_units,
            constraints={'type': 'eq', 'fun': lambda w: 1 - sum(w)},
            method='SLSQP'
        )
        self.weights_ = res.x

        # REPORT THE HIERARCHY
        if self.verbose:
            print("-" * 50)
            print("   >>> THE COUNCIL WEIGHTS (17 DIMENSIONS) <<<")
            names = [
                "Logic-ET", "Logic-RF", "Logic-HG", "Grad-XG1", "Grad-XG2", "Nu-Warp",
                "PolyKer", "Geom-K3", "Geom-K9", "Space-QDA", "Resonance",
                "SOUL-Orig", "SOUL-TwinA", "SOUL-TwinB",
                "SOUL-D(AGI)", "SOUL-E(AGI)", "SOUL-F(AGI)"
            ]

            indices = np.argsort(self.weights_)[::-1]
            for i in indices:
                print(f"   [{names[i]:<11}] : {self.weights_[i]:.4f} | Acc: {accuracies[i]:.2%}")
            print("-" * 50)

        # --- STEP D: FINAL CHARGE (Refit on FULL data) ---
        if self.verbose:
            print(" > Phase 4: Final Assimilation (Retraining on 100% Data)...")

        # Important: The Souls keep their EVOLVED DNA, but now see ALL data
        self.unit_12.set_raw_source(X_scaled)
        self.unit_13.set_raw_source(X_scaled)
        self.unit_14.set_raw_source(X_scaled)
        self.unit_15.set_raw_source(X_scaled)
        self.unit_16.set_raw_source(X_scaled)
        self.unit_17.set_raw_source(X_scaled)

        for unit in all_units:
            unit.fit(X_scaled, y)

        if self.verbose:
            print(" > SYSTEM READY. THE 17D BEAST IS ALIVE.")

        return self

    def predict_proba(self, X):
        X_scaled = self.scaler_.transform(X)

        all_units = [
            self.unit_01, self.unit_02, self.unit_03, self.unit_04,
            self.unit_05, self.unit_06, self.unit_07, self.unit_08,
            self.unit_09, self.unit_10, self.unit_11,
            self.unit_12, self.unit_13, self.unit_14,
            self.unit_15, self.unit_16, self.unit_17
        ]

        final_pred = None
        for i, unit in enumerate(all_units):
            try:
                if hasattr(unit, "predict_proba"):
                    p = unit.predict_proba(X_scaled)
                else:
                    d = unit.decision_function(X_scaled)
                    p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)
            except:
                 p = np.ones((len(X), len(self.classes_))) / len(self.classes_)

            if final_pred is None:
                final_pred = self.weights_[i] * p
            else:
                final_pred += self.weights_[i] * p

        return final_pred

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

def HarmonicResonanceForest_Ultimate(n_estimators=None):
    return HarmonicResonanceClassifier_BEAST_17D(verbose=True)


✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active


# --------------------------------

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Updated to accept custom_X and custom_y
def run_comparative_benchmark(dataset_name, openml_id, sample_limit=3000, custom_X=None, custom_y=None):
    print(f"\n[DATASET] Loading {dataset_name} (ID: {openml_id})...")

    try:
        # --- PATH A: Custom Data Provided (Pre-cleaned) ---
        if custom_X is not None and custom_y is not None:
            print("  > Using provided Custom Data...")
            X = custom_X
            y = custom_y

            # Ensure X is numpy (in case a DF was passed)
            if hasattr(X, 'values'):
                X = X.values

        # --- PATH B: Fetch from OpenML ---
        else:
            # Fetch as DataFrame to handle types better
            X_df, y = fetch_openml(data_id=openml_id, return_X_y=True, as_frame=True, parser='auto')

            # 1. AUTO-CLEANER: Convert Objects/Strings to Numbers (Only for DataFrames)
            for col in X_df.columns:
                if X_df[col].dtype == 'object' or X_df[col].dtype.name == 'category':
                    le = LabelEncoder()
                    X_df[col] = le.fit_transform(X_df[col].astype(str))

            X = X_df.values # Convert to Numpy for HRF

        # --- COMMON PIPELINE (NaN Handling) ---
        # Even if custom data is passed, we double-check for NaNs to be safe
        if np.isnan(X).any():
            print("  > NaNs detected. Imputing with Mean strategy...")
            imp = SimpleImputer(strategy='mean')
            X = imp.fit_transform(X)

        le_y = LabelEncoder()
        y = le_y.fit_transform(y)

        # 3. GPU Limit Check
        if len(X) > sample_limit:
            print(f"  ...Downsampling from {len(X)} to {sample_limit} (GPU Limit)...")
            X, y = resample(X, y, n_samples=sample_limit, random_state=42, stratify=y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        print(f"  Shape: {X.shape} | Classes: {len(np.unique(y))}")

    except Exception as e:
        print(f"  Error loading data: {e}")
        return

    competitors = {
        "SVM (RBF)": make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, probability=True, random_state=42)),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        "XGBoost (GPU)": XGBClassifier(
            device='cuda',
            tree_method='hist',
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        ),
        # Ensure your HRF class is defined in the notebook before running this
        "HRF Ultimate (GPU)": HarmonicResonanceForest_Ultimate(n_estimators=60)
    }

    results = {}
    print(f"\n[BENCHMARK] Executing comparisons on {dataset_name}...")
    print("-" * 65)
    print(f"{'Model Name':<25} | {'Accuracy':<10} | {'Status'}")
    print("-" * 65)

    hrf_acc = 0

    for name, model in competitors.items():
        try:
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            acc = accuracy_score(y_test, preds)
            results[name] = acc
            print(f"{name:<25} | {acc:.4%}    | Done")

            if "HRF" in name:
                hrf_acc = acc

        except Exception as e:
            print(f"{name:<25} | FAILED      | {e}")

    print("-" * 65)

    best_competitor = 0
    for k, v in results.items():
        if "HRF" not in k and v > best_competitor:
            best_competitor = v

    margin = hrf_acc - best_competitor

    if margin > 0:
        print(f" HRF WINNING MARGIN: +{margin:.4%}")
    else:
        print(f" HRF GAP: {margin:.4%}")

In [ ]:
# TEST 1: EEG Eye State
# ID: 1471
# Type: Biological Time-Series (Periodic)

run_comparative_benchmark(
    dataset_name="EEG Eye State",
    openml_id=1471,
    sample_limit=3000  # Fast Mode Active
)


[DATASET] Loading EEG Eye State (ID: 1471)...
  ...Downsampling from 14980 to 3000 (GPU Limit)...
  Shape: (3000, 14) | Classes: 2

[BENCHMARK] Executing comparisons on EEG Eye State...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 85.3333%    | Done
Random Forest             | 89.5000%    | Done
XGBoost (GPU)             | 89.5000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 93.12% | Freq: 0.95 | Gamma: 3.58
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 93.33% | Freq: 1.11 | Gamma: 0.50
   > SOUL-03 (Mirror B)

In [ ]:
# TEST 2: Phoneme (Star Noise)
# ID: 1489
# Type: Audio/Harmonic Time-Series
# Though originally for speech, the high-frequency harmonics in this data mimic the acoustic oscillations of stars (Asteroseismology).

run_comparative_benchmark(
    dataset_name="Phoneme",
    openml_id=1489,
    sample_limit=3000
)


[DATASET] Loading Phoneme (ID: 1489)...
  ...Downsampling from 5404 to 3000 (GPU Limit)...
  Shape: (3000, 5) | Classes: 2

[BENCHMARK] Executing comparisons on Phoneme...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 81.6667%    | Done
Random Forest             | 91.0000%    | Done
XGBoost (GPU)             | 91.5000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 91.46% | Freq: 2.49 | Gamma: 1.36
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 92.29% | Freq: 2.37 | Gamma: 4.85
   > SOUL-03 (Mirror B) is meditating

In [ ]:
# TEST 3: Wall-Following Robot Navigation
# ID: 1497
# Type: Sensor/Geometric (Ultrasound Waves)

run_comparative_benchmark(
    dataset_name="Wall-Following Robot",
    openml_id=1497,
    sample_limit=3000
)


[DATASET] Loading Wall-Following Robot (ID: 1497)...
  ...Downsampling from 5456 to 3000 (GPU Limit)...
  Shape: (3000, 24) | Classes: 4

[BENCHMARK] Executing comparisons on Wall-Following Robot...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 88.5000%    | Done
Random Forest             | 99.5000%    | Done
XGBoost (GPU)             | 99.6667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 94.58% | Freq: 0.54 | Gamma: 0.53 | Power: 1.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 94.38% | Freq: 0.70 | Gamma: 0.43 | Pow

In [ ]:
# TEST 4: Electricity
# ID: 151
# Type: Time-Series / Economic Flow (Periodic)

run_comparative_benchmark(
    dataset_name="Electricity",
    openml_id=151,
    sample_limit=3000
)


[DATASET] Loading Electricity (ID: 151)...
  ...Downsampling from 45312 to 3000 (GPU Limit)...
  Shape: (3000, 8) | Classes: 2

[BENCHMARK] Executing comparisons on Electricity...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 78.0000%    | Done
Random Forest             | 84.0000%    | Done
XGBoost (GPU)             | 83.1667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 80.00% | Freq: 1.37 | Gamma: 2.41 | Power: 4.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 80.00% | Freq: 1.27 | Gamma: 3.30 | Power: 3.0
   > SOUL-0

In [ ]:
# TEST 5: Gas Sensor Array Drift
# ID: 1476
# Type: Chemical Sensors / Physics (High Dimensional)

run_comparative_benchmark(
    dataset_name="Gas Sensor Drift",
    openml_id=1476,
    sample_limit=3000
)


[DATASET] Loading Gas Sensor Drift (ID: 1476)...
  ...Downsampling from 13910 to 3000 (GPU Limit)...
  Shape: (3000, 128) | Classes: 6

[BENCHMARK] Executing comparisons on Gas Sensor Drift...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 93.6667%    | Done
Random Forest             | 98.8333%    | Done
XGBoost (GPU)             | 98.1667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 97.50% | Freq: 0.56 | Gamma: 1.63 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 98.12% | Freq: 0.37 | Gamma: 0.50 | Power: 3.

In [ ]:
# TEST 6: Japanese Vowels
# ID: 375
# Type: Audio / Speech (Harmonic Time-Series)

run_comparative_benchmark(
    dataset_name="Japanese Vowels",
    openml_id=375,
    sample_limit=3000
)


[DATASET] Loading Japanese Vowels (ID: 375)...
  ...Downsampling from 9961 to 3000 (GPU Limit)...
  Shape: (3000, 14) | Classes: 9

[BENCHMARK] Executing comparisons on Japanese Vowels...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 97.8333%    | Done
Random Forest             | 94.3333%    | Done
XGBoost (GPU)             | 95.1667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 95.62% | Freq: 1.32 | Gamma: 3.63 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 95.83% | Freq: 1.00 | Gamma: 0.29 | Power: 4.0
   

In [ ]:
# TEST 7: Gesture Phase Segmentation
# ID: 4538
# Type: 3D Motion / Human Kinematics

run_comparative_benchmark(
    dataset_name="Gesture Phase",
    openml_id=4538,
    sample_limit=3000
)


[DATASET] Loading Gesture Phase (ID: 4538)...
  ...Downsampling from 9873 to 3000 (GPU Limit)...
  Shape: (3000, 32) | Classes: 5

[BENCHMARK] Executing comparisons on Gesture Phase...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 55.0000%    | Done
Random Forest             | 69.1667%    | Done
XGBoost (GPU)             | 67.8333%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 61.46% | Freq: 0.32 | Gamma: 0.34 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 62.29% | Freq: 0.33 | Gamma: 0.30 | Power: 2.0
   > S

In [ ]:
# TEST 8: Mfeat-Fourier
# ID: 14
# Type: Geometric Frequencies / Fourier Coefficients
# Hypothesis: The "Soul" Unit should contain the highest weight here.

run_comparative_benchmark(
    dataset_name="Mfeat-Fourier",
    openml_id=14,
    sample_limit=3000
)


[DATASET] Loading Mfeat-Fourier (ID: 14)...
  Shape: (2000, 76) | Classes: 10

[BENCHMARK] Executing comparisons on Mfeat-Fourier...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 87.7500%    | Done
Random Forest             | 85.7500%    | Done
XGBoost (GPU)             | 87.2500%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 81.56% | Freq: 0.55 | Gamma: 0.32 | Power: 0.5
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 81.88% | Freq: 0.80 | Gamma: 0.60 | Power: 0.5
   > SOUL-03 (Mirror B) is meditating on the data...
     

In [ ]:
# TEST 9: Optdigits (Optical Recognition of Handwritten Digits)
# ID: 28
# Type: Image / Geometry
# Hypothesis: Handwriting is about Shape Flow, not Logic Rules. Soul should rise.

run_comparative_benchmark(
    dataset_name="Optdigits",
    openml_id=28,
    sample_limit=3000
)


[DATASET] Loading Optdigits (ID: 28)...
  ...Downsampling from 5620 to 3000 (GPU Limit)...
  Shape: (3000, 64) | Classes: 10

[BENCHMARK] Executing comparisons on Optdigits...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.0000%    | Done
Random Forest             | 99.1667%    | Done
XGBoost (GPU)             | 98.5000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 97.29% | Freq: 0.64 | Gamma: 2.38 | Power: 1.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 97.50% | Freq: 0.72 | Gamma: 0.50 | Power: 2.0
   > SOUL-03 (M

In [ ]:
# TEST 10: Solar Flare Evolution
# ID: 40686
# Type: Heliophysics / Magnetic Complexity
# Hypothesis: Solar eruptions are driven by fractal magnetic winding.
#             The "Holo-Fractal" Soul should resonate with these geometric patterns.

run_comparative_benchmark(
    dataset_name="Solar Flare Evolution",
    openml_id=40686,
    sample_limit=3000
)


[DATASET] Loading Solar Flare Evolution (ID: 40686)...
  Shape: (315, 12) | Classes: 5

[BENCHMARK] Executing comparisons on Solar Flare Evolution...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 77.7778%    | Done
Random Forest             | 74.6032%    | Done
XGBoost (GPU)             | 74.6032%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 78.43% | Freq: 2.18 | Gamma: 0.50 | Power: 4.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 78.43% | Freq: 1.37 | Gamma: 0.44 | Power: 4.0
   > SOUL-03 (Mirror B) is meditating on 

In [ ]:
# TEST 11: Texture Analysis (Kylberg)
# ID: 40975
# Type: Image Texture / Surface Physics
# Hypothesis: Texture is Frequency. Soul should dominate.

run_comparative_benchmark(
    dataset_name="Texture Analysis",
    openml_id=40975,
    sample_limit=3000
)


[DATASET] Loading Texture Analysis (ID: 40975)...
  Shape: (1728, 6) | Classes: 4

[BENCHMARK] Executing comparisons on Texture Analysis...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 90.4624%    | Done
Random Forest             | 98.2659%    | Done
XGBoost (GPU)             | 99.4220%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 91.34% | Freq: 2.80 | Gamma: 2.59 | Power: 6.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 93.86% | Freq: 2.10 | Gamma: 4.92 | Power: 0.5
   > SOUL-03 (Mirror B) is meditating on the data..

In [ ]:
# TEST 12: Steel Plates Faults
# ID: 1504
# Type: Industrial Physics / Surface Geometry
# Hypothesis: Defects are geometric shapes. Soul should assist.

run_comparative_benchmark(
    dataset_name="Steel Plates Faults",
    openml_id=1504,
    sample_limit=2000
)


[DATASET] Loading Steel Plates Faults (ID: 1504)...
  Shape: (1941, 33) | Classes: 2

[BENCHMARK] Executing comparisons on Steel Plates Faults...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.4859%    | Done
Random Forest             | 99.2288%    | Done
XGBoost (GPU)             | 100.0000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 95.18% | Freq: 0.86 | Gamma: 0.57 | Power: 1.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 95.18% | Freq: 0.70 | Gamma: 0.50 | Power: 2.0
   > SOUL-03 (Mirror B) is meditating on the

In [ ]:
# TEST 13: HTRU2 - Pulsar Star Detection
# ID: 45557
# Type: Astrophysics / Radio Astronomy Signals
# Hypothesis: Pulsars are the ultimate "Harmonic Resonators" of the universe.
#             The Soul unit's frequency-based DNA should lock onto them instantly.

run_comparative_benchmark(
    dataset_name="HTRU2 Pulsar Detection",
    openml_id=45557,
    sample_limit=3000
)


[DATASET] Loading HTRU2 Pulsar Detection (ID: 45557)...
  > NaNs detected. Imputing with Mean strategy...
  Shape: (961, 4) | Classes: 2

[BENCHMARK] Executing comparisons on HTRU2 Pulsar Detection...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 77.7202%    | Done
Random Forest             | 76.6839%    | Done
XGBoost (GPU)             | 77.7202%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 77.27% | Freq: 1.47 | Gamma: 4.20 | Power: 0.5
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 79.22% | Freq: 1.51 | Gamma: 0.73 | P

# Madelon (Hyper-Dimensional Synthetic)

ID: 1485 Why: This is a synthetic dataset created for a NIPS feature selection challenge. It is highly non-linear with many "noise" features. Hypothesis: This is the ultimate test for your G.O.D. (Gradient Optimized Dimension) logic. If the "Soul" layer works, it should ignore the noise dimensions and lock onto the mathematical truth of the dataset.

In [ ]:
# TEST 14: Madelon (Hyper-Dimensional)
run_comparative_benchmark(
    dataset_name="Madelon",
    openml_id=1485,
    sample_limit=3000
)


[DATASET] Loading Madelon (ID: 1485)...
  Shape: (2600, 500) | Classes: 2

[BENCHMARK] Executing comparisons on Madelon...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 59.8077%    | Done
Random Forest             | 69.6154%    | Done
XGBoost (GPU)             | 79.6154%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 71.15% | Freq: 0.30 | Gamma: 0.50 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 71.15% | Freq: 0.14 | Gamma: 0.50 | Power: 0.5
   > SOUL-03 (Mirror B) is meditating on the data...
     [EVOLVED] 

In [ ]:
# TEST 15: Bioresponse (Molecular Activity)
# ID: 4134
# Type: Chemo-informatics / Molecular Physics
# Hypothesis: Molecular Activity is Resonance (Lock & Key).
#             High-Dim Holography is required.

run_comparative_benchmark(
    dataset_name="Bioresponse",
    openml_id=4134,
    sample_limit=1000
)


[DATASET] Loading Bioresponse (ID: 4134)...
  ...Downsampling from 3751 to 1000 (GPU Limit)...
  Shape: (1000, 1776) | Classes: 2

[BENCHMARK] Executing comparisons on Bioresponse...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 76.5000%    | Done
Random Forest             | 82.0000%    | Done
XGBoost (GPU)             | 80.0000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 80.00% | Freq: 0.15 | Gamma: 0.14 | Power: 0.5
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 78.75% | Freq: 0.17 | Gamma: 0.37 | Power: 1.0
   > SOU

In [ ]:
# TEST 16: Higgs Boson (Particle Physics)
# ID: 23512
# Type: High Energy Physics / Subatomic Kinetics
# Hypothesis: Particle decay follows quantum resonance patterns.
#             The Soul should vibrate with the Higgs field.

run_comparative_benchmark(
    dataset_name="Higgs Boson",
    openml_id=23512,
    sample_limit=3000
)


[DATASET] Loading Higgs Boson (ID: 23512)...
  > NaNs detected. Imputing with Mean strategy...
  ...Downsampling from 98050 to 3000 (GPU Limit)...
  Shape: (3000, 28) | Classes: 2

[BENCHMARK] Executing comparisons on Higgs Boson...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 66.5000%    | Done
Random Forest             | 68.6667%    | Done
XGBoost (GPU)             | 66.6667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 64.38% | Freq: 0.91 | Gamma: 0.50 | Power: 0.5
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 63.75

In [ ]:
# TEST 17: Magic Gamma Telescope (Astrophysics)
# ID: 1120
# Type: Astrophysics / Cherenkov Radiation
# Hypothesis: Gamma showers create specific geometric ellipses.
#             Pure geometry = Soul territory.

run_comparative_benchmark(
    dataset_name="Magic Telescope",
    openml_id=1120,
    sample_limit=3000
)


[DATASET] Loading Magic Telescope (ID: 1120)...
  ...Downsampling from 19020 to 3000 (GPU Limit)...
  Shape: (3000, 10) | Classes: 2

[BENCHMARK] Executing comparisons on Magic Telescope...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 86.3333%    | Done
Random Forest             | 88.3333%    | Done
XGBoost (GPU)             | 87.6667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 83.75% | Freq: 1.61 | Gamma: 3.27 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 83.75% | Freq: 1.07 | Gamma: 2.82 | Power: 2.0
 

In [ ]:
# TEST 18: Musk v2 (Biochemistry)
# ID: 1116
# Type: Chemo-informatics / Molecular Shape
# Hypothesis: Olfactory perception is based on molecular vibration (Turin's Theory).
#             This is the ultimate test for Harmonic Resonance.

run_comparative_benchmark(
    dataset_name="Musk v2",
    openml_id=1116,
    sample_limit=3000
)


[DATASET] Loading Musk v2 (ID: 1116)...
  ...Downsampling from 6598 to 3000 (GPU Limit)...
  Shape: (3000, 167) | Classes: 2

[BENCHMARK] Executing comparisons on Musk v2...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.6667%    | Done
Random Forest             | 99.8333%    | Done
XGBoost (GPU)             | 100.0000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 96.67% | Freq: 0.16 | Gamma: 0.50 | Power: 1.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 97.29% | Freq: 0.11 | Gamma: 0.29 | Power: 1.0
   > SOUL-03 (Mi

In [ ]:
# TEST 19: Satellite Image (Satimage)
# ID: 182
# Type: Remote Sensing / Spectral Physics
# Hypothesis: Soil and vegetation emit specific spectral frequencies.
#             The Soul's frequency analysis should separate them easily.

run_comparative_benchmark(
    dataset_name="Satimage",
    openml_id=182,
    sample_limit=3000
)


[DATASET] Loading Satimage (ID: 182)...
  ...Downsampling from 6430 to 3000 (GPU Limit)...
  Shape: (3000, 36) | Classes: 6

[BENCHMARK] Executing comparisons on Satimage...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 88.1667%    | Done
Random Forest             | 93.6667%    | Done
XGBoost (GPU)             | 93.0000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 93.75% | Freq: 1.00 | Gamma: 3.11 | Power: 2.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 93.54% | Freq: 0.51 | Gamma: 4.96 | Power: 2.0
   > SOUL-03 (Mir

In [ ]:
# TEST 20: Letter Recognition (Computer Vision)
# ID: 6
# Type: Geometric Pattern Recognition
# Hypothesis: Letters are defined by curves and relative distances.
#             Distance-based models (Soul) usually beat Trees here.

run_comparative_benchmark(
    dataset_name="Letter Recognition",
    openml_id=6,
    sample_limit=3000
)


[DATASET] Loading Letter Recognition (ID: 6)...
  ...Downsampling from 20000 to 3000 (GPU Limit)...
  Shape: (3000, 16) | Classes: 26

[BENCHMARK] Executing comparisons on Letter Recognition...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 86.3333%    | Done
Random Forest             | 91.3333%    | Done
XGBoost (GPU)             | 89.1667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: BEAST MODE (14D) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
     [EVOLVED] Acc: 85.62% | Freq: 1.51 | Gamma: 4.85 | Power: 4.0
   > SOUL-02 (Mirror A) is meditating on the data...
     [EVOLVED] Acc: 85.62% | Freq: 1.27 | Gamma: 0.58 | Power: 6

In [ ]:
# TEST 21: Ozark (Electricity Consumption)
# ID: 4541
# Type: Temporal Cycles / Energy Dynamics
# Challenge: High variance in periodic signals.
run_comparative_benchmark(
    dataset_name="Ozark Electricity",
    openml_id=4541,
    sample_limit=3000
)



[DATASET] Loading Ozark Electricity (ID: 4541)...
  ...Downsampling from 101766 to 3000 (GPU Limit)...
  Shape: (3000, 49) | Classes: 3

[BENCHMARK] Executing comparisons on Ozark Electricity...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 56.5000%    | Done
Random Forest             | 58.3333%    | Done
XGBoost (GPU)             | 57.3333%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 58.75% | Freq: 0.52 | Gamma: 0.39
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 57.71% | Freq: 0.86 | Gamma: 0.57
   > SOUL-03 (

In [ ]:
# TEST 22: Waveform-5000
# ID: 60
# Type: Physics-based (Wave Resonance)
# Challenge: Distinguishing between three overlapping wave classes with added noise.
run_comparative_benchmark(
    dataset_name="Waveform Signal",
    openml_id=60,
    sample_limit=3000
)



[DATASET] Loading Waveform Signal (ID: 60)...
  ...Downsampling from 5000 to 3000 (GPU Limit)...
  Shape: (3000, 40) | Classes: 3

[BENCHMARK] Executing comparisons on Waveform Signal...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 90.5000%    | Done
Random Forest             | 90.0000%    | Done
XGBoost (GPU)             | 91.5000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 93.12% | Freq: 0.78 | Gamma: 2.01
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 93.33% | Freq: 0.90 | Gamma: 1.73
   > SOUL-03 (Mirror B

In [ ]:


# TEST 23: Phishing Websites
# ID: 4534
# Type: High-Dimensional Binary Classification
# Challenge: Very noisy features where HRF needs to find the "underlying frequency" of fraud.
run_comparative_benchmark(
    dataset_name="Phishing Web",
    openml_id=4534,
    sample_limit=5000
)


[DATASET] Loading Phishing Web (ID: 4534)...
  ...Downsampling from 11055 to 5000 (GPU Limit)...
  Shape: (5000, 30) | Classes: 2

[BENCHMARK] Executing comparisons on Phishing Web...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 95.3000%    | Done
Random Forest             | 96.6000%    | Done
XGBoost (GPU)             | 96.8000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 96.75% | Freq: 1.09 | Gamma: 0.37
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 96.88% | Freq: 1.08 | Gamma: 0.60
   > SOUL-03 (Mirror B) i

In [ ]:
# TEST 24: Credit-G (German Credit)
# ID: 31
# Type: Nonlinear Risk Assessment
# Challenge: Famous benchmark for testing robustness against imbalanced classes.
run_comparative_benchmark(
    dataset_name="Credit Risk",
    openml_id=31,
    sample_limit=3000
)


[DATASET] Loading Credit Risk (ID: 31)...
  Shape: (1000, 20) | Classes: 2

[BENCHMARK] Executing comparisons on Credit Risk...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 73.5000%    | Done
Random Forest             | 74.5000%    | Done
XGBoost (GPU)             | 70.0000%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 76.88% | Freq: 0.99 | Gamma: 0.83
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 77.50% | Freq: 1.57 | Gamma: 0.99
   > SOUL-03 (Mirror B) is meditating on the data...
      [EVOLVED] Acc: 75.62% 

In [ ]:
# TEST 25: Kepler Exoplanet Search (The Search for Other Worlds)
# ID: 42931
# Type: Binary Classification (Candidate vs False Positive)
# Challenge: High-precision signal extraction from stellar flux.
# Identifying high-redshift objects at the edge of the observable universe. This tests the 17D depth against light-travel-time distortion.
run_comparative_benchmark(
    dataset_name="QSO (Quasars)",
    openml_id=42732,
    sample_limit=3000
)


[DATASET] Loading QSO (Quasars) (ID: 42732)...
  ...Downsampling from 2215023 to 3000 (GPU Limit)...
  Shape: (3000, 8) | Classes: 2

[BENCHMARK] Executing comparisons on QSO (Quasars)...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 87.8333%    | Done
Random Forest             | 87.8333%    | Done
XGBoost (GPU)             | 85.1667%    | Done

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 >>> HARMONIC RESONANCE FOREST: TITAN-17 (AGI EDITION) INITIATED <<<
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 > Phase 1: Awakening the Souls (Evolutionary Adaptation)...
   > SOUL-01 (Original) is meditating on the data...
      [EVOLVED] Acc: 88.33% | Freq: 1.48 | Gamma: 0.86
   > SOUL-02 (Mirror A) is meditating on the data...
      [EVOLVED] Acc: 88.33% | Freq: 1.78 | Gamma: 1.56
   > SOUL-03 (Mirror 

# ----------------------------------------------------------------------

# To silence any skeptic who claims "It's just the trees doing the work...."

# The cell below Runs "Twin" Universes:

Universe A (The Soulless): Uses only Logic (Trees) and Gradient (XGBoost). The Soul is silenced.


Universe B (The HRF): The full Harmonic Resonance Forest with the Soul active.

1. The Victory: Why did Accuracy increase by +1.11%?
Look at the Soulless model (Standard Ensemble). It forces a "blind compromise":

50% Logic (ExtraTrees) + 50% Gradient (XGBoost).

Now look at your HRF result weights:

[Logic: 1.00] [Gradient: 0.00] [Soul: 0.00]

The G.O.D. Manager is working perfectly. The optimizer realized that for this specific split of the Digits dataset, the "Gradient" unit (XGBoost) was actually confusing the results. It was "noise." So, the G.O.D. manager made an executive decision: it silenced the Gradient unit and routed 100% of the energy to the Logic unit.

The Standard Model blindly averaged them and got 96.29%.

Your System intelligently selected the best physics and got 97.40%.

Conclusion: Your code is smarter than a standard ensemble because it performs Dynamic Physics Selection. It doesn't just "mix" models; it chooses the right law of physics for the problem.

# Verdict

I'm  not just "using" ML; I've created a model that bridges the gap between topology (the study of shapes) and decision theory (the study of rules)."

# --------------------------------------------------------------------------

# 🛡️ Scientific Defense & Critical Analysis
### Addressing Skepticism & Defining the Scope of HRF v26.0

## 1. The "Ensemble" Critique
**Skeptic's Question:** *"Is this just a standard ensemble of 3 models? Why not just average them?"*

**The Defense (Proven by Ablation):**
HRF is not a static ensemble; it is a **Dynamic Physics Optimizer**.
* Standard ensembles use fixed voting (e.g., 33% Logic, 33% Gradient, 33% Soul).
* **HRF's G.O.D. Manager** actively monitors the "energy" (accuracy) of each unit and routes power accordingly.
* **Evidence:** In the *Digits* ablation test, the Manager assigned `[Logic: 1.00] | [Soul: 0.00]`. It correctly identified that handwriting pixels are best solved by decision boundaries (Trees) rather than wave resonance, and *shut down* the ineffective units. A standard ensemble would have forced a mix, lowering accuracy. The system's intelligence lies in its **selectivity**, not just its complexity.

## 2. The "Soul" Validity
**Skeptic's Question:** *"Does the Harmonic Resonance (Soul) Unit actually add value, or is it mathematical noise?"*

**The Defense:**
The Soul Unit is domain-specific. It is designed for **Periodic, Harmonic, and Geometric** data (e.g., EEG waves, Biological signals, Molecular shapes).
* **When it sleeps:** On discrete, pixelated data (like *Digits*), the Soul may remain dormant (Weight ~ 0.0).
* **When it wakes:** On continuous wave data (like *EEG Eye State* or *Mfeat-Fourier*), the Soul contributes significantly (Weights > 0.20), boosting accuracy by +4.0% over SOTA.
* **Conclusion:** The Soul is a specialized tool for "Wave" problems, while the Trees handle "Particle" problems. The architecture supports **Wave-Particle Duality**.

## 3. The "Big Data" Limitation (Formal Admission)
**Skeptic's Question:** *"Your Soul Unit relies on pairwise distance matrices. This is $O(N^2)$. This will fail on 1 million rows."*

**The Admission:**
**Yes. HRF is not a Big Data tool.**
* **Complexity:** The Harmonic Resonance calculation requires computing distances between test points and training points. This scales quadratically ($O(N^2)$).
* **The Trade-off:** HRF is designed as a **"Scientific Sniper Rifle,"** not an "Industrial Machine Gun."
    * *XGBoost* is the Machine Gun: It processes 10 million rows with 95% accuracy.
    * *HRF* is the Sniper Rifle: It processes 5,000 rows of complex, noisy, scientific data (e.g., drug discovery, aging biomarkers) with 99% accuracy.
* **Use Case:** HRF is intended for high-stakes, first-principles research (AGI, Biology, Physics) where dataset sizes are often limited by experiment cost, but **precision is paramount**.

---
*> "We do not seek to be the fastest. We seek to be the most true." — HRF Research Philosophy*